# Tarea 2 Procesamiento de Lenguaje Natural

Estudiante: Sofía Vits
Profesora: María Paz Raveau
Fecha entrega: 3 de agosto 2025

### Introducción

### Desarrollo

#### Carga de librerías

In [1]:
! pip install stanza

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 98.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 46.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12

In [2]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import pandas as pd
import numpy as np
import math
import nltk
import string

remove_punct_map = dict.fromkeys(map(ord, string.punctuation))
pd.set_option('display.max_rows',1000)
pd.set_option('display.max_columns',1000)
from itertools import compress
from nltk import word_tokenize
import re

In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
import stanza
nlp = stanza.Pipeline(lang='es', processors='tokenize,mwt,pos,lemma')

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json


INFO:stanza:Loading these models for language: es (Spanish):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| pos       | combined_charlm   |
| lemma     | combined_nocharlm |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Done loading processors!


#### Carga de archivo y pre-procesamiento

In [5]:
ecqq = pd.read_csv('ecqq.csv', sep='delimiter', on_bad_lines='warn')

/tmp/ipython-input-5-1315084596.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ecqq = pd.read_csv('ecqq.csv', sep='delimiter', on_bad_lines='warn')


In [6]:
ecqq.shape

(10203, 1)

In [7]:
ecqq.head()

,P2_1_B
0,0\tse necesitan pensiones dignas a los adultos...
1,"1\tel acceso a la salud sea de calidad, con di..."
2,"2\t""conprender que el trabajo es una """"activid..."
3,"junto con ello, asimilar a trabajadores honora..."
4,"3\t""- más especialistas en los consultorios, s..."


In [8]:
ecqq = ecqq[[len(x.split( )) >15  for x in ecqq.P2_1_B]]

In [9]:
ecqq.shape

(2565, 1)

In [10]:
from nltk.corpus import stopwords
esp_stop = stopwords.words('spanish')
esp_stop = esp_stop + ['ser','tener','deber','haber','hacer','poder','ir']
esp_stop.remove('estado')
esp_stop.remove('estados')

In [11]:
esp_stop

['de',
 'la',
 'que',
 'el',
 'en',
 'y',
 'a',
 'los',
 'del',
 'se',
 'las',
 'por',
 'un',
 'para',
 'con',
 'no',
 'una',
 'su',
 'al',
 'lo',
 'como',
 'más',
 'pero',
 'sus',
 'le',
 'ya',
 'o',
 'este',
 'sí',
 'porque',
 'esta',
 'entre',
 'cuando',
 'muy',
 'sin',
 'sobre',
 'también',
 'me',
 'hasta',
 'hay',
 'donde',
 'quien',
 'desde',
 'todo',
 'nos',
 'durante',
 'todos',
 'uno',
 'les',
 'ni',
 'contra',
 'otros',
 'ese',
 'eso',
 'ante',
 'ellos',
 'e',
 'esto',
 'mí',
 'antes',
 'algunos',
 'qué',
 'unos',
 'yo',
 'otro',
 'otras',
 'otra',
 'él',
 'tanto',
 'esa',
 'estos',
 'mucho',
 'quienes',
 'nada',
 'muchos',
 'cual',
 'poco',
 'ella',
 'estar',
 'estas',
 'algunas',
 'algo',
 'nosotros',
 'mi',
 'mis',
 'tú',
 'te',
 'ti',
 'tu',
 'tus',
 'ellas',
 'nosotras',
 'vosotros',
 'vosotras',
 'os',
 'mío',
 'mía',
 'míos',
 'mías',
 'tuyo',
 'tuya',
 'tuyos',
 'tuyas',
 'suyo',
 'suya',
 'suyos',
 'suyas',
 'nuestro',
 'nuestra',
 'nuestros',
 'nuestras',
 'vuestro'

#### Lemmatización

In [12]:
def lemmatizer(text):
  doc = nlp(text)
  return ' '.join([word.lemma for sent in doc.sentences for word in sent.words if word.lemma not in esp_stop])

In [13]:
ecqq['lemma'] = [lemmatizer(x) for x in ecqq['P2_1_B']]

In [37]:
ecqq['lemma'] = ecqq['lemma'].str.lower()
ecqq['lemma'] = ecqq['lemma'].str.translate(str.maketrans(' ', ' ', string.digits))
ecqq['lemma'] = ecqq['lemma'].str.translate(str.maketrans(' ', ' ', string.punctuation))
ecqq['lemma'] = ecqq['lemma'].str.replace('\n',' ')
ecqq['lemma'] = ecqq['lemma'].str.strip()
ecqq['lemma'] = ecqq['lemma'].str.replace('\d+', '')
ecqq['lemma'] = ecqq['lemma'].str.replace("°", "")
ecqq['lemma'] = ecqq['lemma'].str.replace('"-', "")
ecqq['lemma'] = ecqq['lemma'].str.replace('"', "")
ecqq['lemma'] = ecqq['lemma'].str.replace('•', "")

ecqq = ecqq.reset_index(drop=True)

In [41]:
ecqq = ecqq[['P2_1_B','lemma']]

In [42]:
ecqq.shape

(2565, 2)

In [40]:
ecqq.to_csv('ecqq_preprocesado.csv',sep='\t', index=False)

### Conclusiones